In [ ]:
from logging import exception
#@title Step -1: Mount drive
#@markdown Run this cell. If prompted, press "Connect to Google Drive" and select your Google account.
#@markdown Then, under the folder icon 📁 on the left panel, you should see the folder **drive** appear.
from google.colab import drive
from IPython.display import display, Markdown, HTML
import os, sys

%load_ext autoreload
%autoreload 2
try:
  drive.mount('/content/drive', force_remount=False)
  # os.chdir('/content/drive/My Drive/DLE-Feb23/Projects')
  sys.path.append('/content/drive/My Drive/DLE-Feb23/Projects')
  os.chdir('/content/drive/MyDrive/Colab Notebooks/')
  display("⭐ Mounted successfully!")
except:
  display(HTML('<span style="color:red">An error occurred. Try again!</span>'))


Mounted at /content/drive


'⭐ Mounted successfully!'

In [ ]:
#@title Step 0: Import Packages
%%capture
!pip install openai
!pip install scikit-learn
!pip install matplotlib
!pip install transformers

import json
import pandas as pd
import openai
import numpy as np
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from dle_utils.dle_utils import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import transformers
import ast
import pprint

## Step 1: Accessing Open AI's API

Before starting this assignment, you'll need to do the following steps:

1. [Create an account on OpenAI](https://auth0.openai.com/u/signup/identifier?state=hKFo2SBzQm1sd2pOTUJ2SG9sSHBBdUU1bGNpdllveFhrWW8wc6Fur3VuaXZlcnNhbC1sb2dpbqN0aWTZIHctYmtFZ3h0ck9saDNVWFhUTnRmckM5azd0RjUtMjMwo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q) (if you have not done so already).

2. In the upper right hand corner, click on your profile icon and select  **View API keys**. On the API keys page, copy your API key into the cell below.

<img src="https://drive.google.com/uc?id=1ctYY7b1TqauMobC6J-Vm_VGBF9nv_4XI" width=800/>

You will only see this key once, so make sure to copy it in a safe place!

3. On the account dropdown, go to **Manage account** to make sure you have access to credits in your account. If you started a free trial, you should have $5.00. If you're like me and your credits have expired/run out, you may want to create a new account! (Otherwise, ChatGPT API access is quite cheap, so you may want to consider adding payment information)

**Remember to remove your API key from the homework before submitting it!**


In [ ]:
# Put down your API key here

openai.api_key = ...

## Deep learning APIs

In a not so distance past, loading and using deep learning models was a laborious and difficult task, involving lots of helper functions and niche knowledge about how models worked.

Now, thanks to the democratization of AI, the most powerful models are available through APIs. In this assignment, we'll give a quick tutorial on the most recent and exciting API: Open AI's ChatGPT API. Then, we'll go through a real-world example on how they might be used to help you in your own work.

### Using OpenAI's API

OpenAI's API is really accessible via Python. There are two basic modes in this API.

1. ```openai.Embedding```
2. ```openai.ChatCompletion```

Roughly speaking, these two modes are just the two levels of abstraction. At the first level, the Embedding API gives you the most raw form of the model -- the embedding vectors of a given text. Next, the ChatCompletion API is a higher level version of the embedding API, where the model has been trained to interpret instructions. (By the way, there are other modules like [Audio](https://platform.openai.com/docs/guides/speech-to-text) which are really cool, but we wont' cover here!)

## Embeddings

The first use case is in extracting model embeddings. This is identical to the model embeddings used in week 2's assignment. Here, we'll just do a quick demo in terms of using these embeddings for a downstream task.

In the cell below, load ```local_df```. In this, you'll see 1000 genuine Google Local reviews from places in Washington, D.C. Each review is associated with a rating from 1-5.

In [ ]:
local_df = pd.read_csv('../DLE-Feb23/Projects/dle_utils/data/google_local_data.csv', index_col=0)
local_df.head()

Let's say that Google users have a tendency to forget to put a 1-5 star rating next to their review. Can we use the embedding API to predict which rating should be associated with each review?

### Extracting embeddings

Use ```openai.Embedding.create()``` to extract embeddings for each review in ```local_df```.  ```openai.Embedding.create()``` takes in two parameters: ```model``` and ```input```. (You can read embeddings [here](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings)).

For ```model```, use ```"text-embedding-ada-002"```, which is the best, fastest, and cheapest ($0.0004 / 1k tokens).

For ```input```, you just need to enter the text you want to embed as a string.

In the cell below, take the first review from ```local_df``` and extract its embedding.

(Hint: For a Pandas dataframe, you can access the text from a specific row with ```local_df.iloc[i]['text']```, where ```i``` is the index of the row.

In [ ]:
# Populate response with the embedding of the first row
sample_review = ...

# Extract embedding
response = ...

print(response.keys())

In [ ]:
check('4.1.1', response)

The output, ```response```, is accessible as a dictionary.

Find where the embeddings are stored in this dictionary. (Hint: You can get the keys to a dictionary with ```.keys()```). The embeddings will be a list of lists, which we want to convert to a numpy array.

In [ ]:
# Extract the embeddings from the response and convert to a numpy array

emb_arr = ...

In [ ]:
check('4.1.2', emb_arr)

Great! Now that we can extract the embedding for any piece of text, we want to get the embeddings for a subset of ```local_df```.

You'll notice the shape of ```emb_arr``` is ```(1536,)```, which is the number of dimensions each embedding is.

In the cell below, populate the list ```sample_emb``` with the embeddings for the first **20** reviews. Then, convert ```sample_emb``` into a numpy array. The shape of ```sample_emb``` should be ```(20, 1536)```.

This should not take more than a couple of minutes.

(You may get a notice like ```RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.``` If this happens, you may need to wait a few more minutes. It may also help to use a [sleep](https://www.programiz.com/python-programming/time/sleep) timer to keep under the rate limit.)

In [ ]:
# Extract embeddings for 20 reviews in local_df
# Populate sample_emb with a numpy array of embeddings

def get_embedding(input_string):
  ...

sample_emb = ...

In [ ]:
check('4.1.3', sample_emb)

Great! Next, we want to gather emebeddings for all 1000 reviews. If you have an OpenAI API account that is older than 48 hours old and you have your credit card attached, you can query 1000 reviews easily. However, newer accounts are rate limited. If you find yourself rate-limited, just skip past this next part and we'll provide you the embeddings for the full 1000 as a ```.npy``` file.

<img src="https://drive.google.com/uc?id=1exdv5DIvWvw9-0O8xYazttEyiuFcdzDg" width=250/>.

In [ ]:
# Extract embeddings for all reviews in local_df
# Populate local_emb with a numpy array of embeddings
# Do this part if you aren't rate-limited!

def get_embedding(input_string):
  ...

local_emb = ...

In [ ]:
# If you are rate-limited, just run this!

local_emb = np.load('../DLE-Feb23/Projects/dle_utils/data/local_emb.npy')

Now that we have a dataset of 1000 embeddings, let's see if we can train a machine learning model to predict the ratings. We can start with doing a binary prediction of whether the review was a low rating (1, 2, or 3) or a high rating (4 or 5).

The cell below has been given to you to train a basic model on this binary task.

In [ ]:
# Just run this!
y = (local_df['rating'] > 3).astype(int).to_list()
X = local_emb

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=42).fit(X_train, y_train)

print(f'Model AUC: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

Not bad, right? An AUC of 0.95 is pretty impressive.

Even though OpenAI's API is pretty cheap, using it across a lot of queries could still cost us some serious dough. Fortunately, free versions of large language models exist. Even better news is that they are incredibly easy to use. Run the line below to load up one of these models.

In [ ]:
from transformers import pipeline
pipeline = pipeline('feature-extraction', model='xlnet-base-cased')

The function ```pipeline``` takes in a string and returns the embedding. However, it will give one embedding for each token, so you'll have to take the average over all tokens like you did in the week 2 assignment.

One advantage of using Huggingface is that it's totally free -- no rate limits :)

In the cell below, extract all the embeddings using ```pipeline``` and store the embeddings as a numpy array in ```local_emb_hf``` (hf stands for huggingface). After doing so, run the machine learning model and see how well these embeddings perform on this task. (It might take a couple minutes!)


In [ ]:
# Extract embeddings for all reviews in local_df
# Populate local_emb_hf with a numpy array of embeddings

local_emb_hf = ...

In [ ]:
check('4.1.4', local_emb_hf)

In [ ]:
y = (local_df['rating'] > 3).astype(int).to_list()
X = local_emb_hf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=42).fit(X_train, y_train)

print(f'Model AUC: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

If you did this right, you should be seeing an AUC of around 0.89. It's not bad, but certaintly not as good as 0.95. As you can see, performance comes with a price.

If you'd like, you can try a couple other models to see how well they do. [List of huggingface models](https://huggingface.co/models).

**Challenge question: Can you find a huggingface model that performs at least 0.90 AUC?**

At the end of the day, whether you use OpenAI's API or a free model from huggingface, the choice just depends on what task you need them for.

___

But what about tasks which we don't have a label for? Suppose we wanted to get the predicted reviews, but we don't have training data? A few years ago, we would have been stuck trying to gather more data. However, now with OpenAI's chat completion API, we may be able to do more than we think.

## Chat Completion

Chat completion provides a whole new dimension of functionality on large language models. Whereas embeddings are hard to interpret machine language, the chat completion API allows us to give the model instructions on how to interpret the prompt. To illustrate this, we will be working with real reviews of restaurants from Google. Run the cell below to read in the data.

In [ ]:
review_df = pd.read_csv('../DLE-Feb23/Projects/dle_utils/data/review_df_cols.csv', index_col=[0])
review_df.head()

You'll notice three columns: ```business_id, review_text, rating```. The business is a unique id associated with each restaurant. The rating is a score from 1-5, and the review_text is, well, the review text.

Rather than try to predict ratings, we're going to try something a little more complicated. Imagine you're working for a restaurant recommendation company, and this user feedback comes in. Your manager wants you to solve this problem and you're tasked with finding the answer using deep learning:

**I love using your app to find restaurants, but each time I get to a restaurant, I don't know what to order. I know that other users write reviews about specific menu items, but it's a time-consuming process to go through each review to find them. I wish there was a quick summary of the reviews for each menu item!**

Let's break down this problem into a couple steps:

1. We want to collect all the reviews for a specific restaurant. For example, we can look at the restaurant with ```business_id=605618c0d335d0abfb415a0f```. Run the cell below to get a sample of the reviews:

In [ ]:
# Just run this!
reviews = review_df[review_df['business_id']=='605618c0d335d0abfb415a0f']['review_text'].to_list()
print('\n'.join(reviews))

2. For each review, we want to ask ChatGPT to extract the reviews for specific menu items. There are a few formats we can get this in, but to make it clear, we can have ChatGPT return it as a json/dictionary.

The idea is to get a lookup table of items, plus their reviews. For example,

```{'chow mein': 'Most users found it delicious, with some thinking it is too spicy', \
    'orange chicken': 'This is a crowd favorite, and has been described as "tangy" and "zesty"'}```

Let's start with one of the reviews:

In [ ]:
# Just run this!
sample_review = reviews[8]
print(sample_review)

Now, we want to get ChatGPT to convert this review to a dictionary:

To query OpenAI's ChatGPT API, use ```openai.ChatCompletion.create()```. You need to pass two parameters:

1. ```model```, which should be 'gpt-3.5-turbo'
2. ```messages```, which should be a list of dictionaries. Each dictionary should contain two entries: ```'role'``` and ```'content'```.

```'role'``` can either be ```'system'```, ```'user'```, or ```'assistant'```.

The role ```'system'``` is meant to be a persistent instruction for the prompt. You can write something like ```'You are an restaurant review writer whose job it is to help people find good menu items. You will return responses is json dictionary format only'```.

The role ```'user'``` is the prompt given. For example, the prompt can be: ```'The following text is a list of restaurant reviews. For each unique menu item mentioned, give a summary of the user reviews for that menu item. Return the output in the form of a json dictionary, where the keys are unique menu items and the values are the summary.'```


In [ ]:
# Run these lines
system_prompt = 'You are an restaurant review writer whose job it is to help \
               people find good menu items. You will return responses is json \
                dictionary format only'

# Feel free to modify this!
prompt = f'The following text is a list of restaurant reviews. \
        For each unique menu item mentioned, give a summary of \
        the user reviews for that menu item. \
        Return the output in the form of a json dictionary, \
        where the keys are unique menu items \
        and the values are the summary. : "{sample_review}"'


The ```.create``` function expects the parameter ```messages``` to be a list, with each element as a dictionary which specifies who (```role```) and what (```content```) is said.

So for example, if we wanted to ask ChatGPT what the first US state is, we could pass in the following into ```messages```:

```messages=[{'role': 'system', 'content': 'You are a helpful tour guide.'}, {'role': 'user', 'content': 'What is the first US state?'}]```

In the cell below, pass the example about US states into ```openai.ChatCompletion.create``` and print out what the output is in ```response```:

In [ ]:
response = ...

print(response)

As you see, the response itself is a dictionary. Here are a couple key things to point out about this response:

1. The actual response is found in "content" under "choices".
2. The "usage" dictionary may be helpful for understanding how many credits you have used per query (ie. for logging purposes). But don't worry about that for this project.

With this information, we are ready to get ChatGPT to convert a restaurant review into a menu item-specific look-up table.

In the cell below, query the API given the variables ```prompt``` and ```system_prompt``` defined for above restaurant reviews:

In [ ]:
response = ...

Next, extract just the content. You will notice it is a string with ```\n``` and ```\```. You can turn this into a Python object using ```ast.literal_eval```. This turns a string of a dictionary into an actual dictionary.

In the cell below, extract the API's response content and convert it into a Python dictionary using ```ast.literal_eval```. (You may need to navigate the ```response``` dictionary a bit to find what you want)

In [ ]:
response_as_dict = ...

print(f' Original review: {sample_review}')
pprint.PrettyPrinter().pprint(response_as_dict)
print('\n')

In [ ]:
check('4.1.5', response_as_dict)

Does this summary make sense?

Next, let's try doing this for multiple reviews.

In the cell below, extend the prompt to include all the reviews in the list ```reviews```. You have the choice of the following:

1. Combine all the reviews into one giant string and combine that with your original prompt

2. Run the prompt once for each review, and then merge the dictionaries into one

3. Do the reviews in chunks (a mix between one and two).

A few hints: (1) If you are running into token limits from OpenAI, you will need to find a way to divide your prompt into smaller bits and combinin them after. (2) Always be as explicit with the API as possible! (Tell it what you are putting in as input, and what you expect to receive as output).



In [ ]:
# Run these lines
system_prompt = ...

prompt = ...

In [ ]:
# Generate response
response = ...
# Store as a dictionary
response_as_dict = ...

print(f' Original review: {"|".join(reviews)}')
pprint.PrettyPrinter().pprint(response_as_dict)
print('\n')

You should have a much longer dictionary, with many more menu items now!

You may have noticed if you called the API multiple times that each call yielded a different result. This is due to the inherent randomness in the model's predictions.

Let's investigate the sources of randomness a little bit here.

In the cell below, generate a second summary of the reviews and store it in the variable ```response_as_dict2```.

This time, add in as a parameter to ```openai.ChatCompletion.create``` the variable ```temperature```. In general, ```temperature``` should be between 0 and 2, with 0 being the least amount of randomness and 2 being the greatest amount of randomness. Choose a temperature of 1.1 and see what the output looks like. (Hint: If the output cannot be interpreted as a dictionary, try running it again, or change the prompt!)



In [ ]:
# Generate response again but with temperature as 1.1
response = ...

# Store as a dictionary
response_as_dict2 = ...

What do you notice about the output? While each run is different, you may see the following: (1) different menu items, (2) different styles of description for each menu item, or (3) a different format alltogether.

In practice, it is good to get a sample of the types of outputs being produced. Another parameter to use is ```n```, which just controls the number of samples you want ChatGPT to produce.

In the cell below, rather than changing temperature, just set ```n=2```.

In [ ]:
# Generate response again but with n=2
response = ...

# Store as a dictionary
response_run1 = ...
response_run2 = ...

Take a look at a specific menu item between the first and the second call. What kinds of differences do you notice? Are the menu items picked up by the API the same?

Feel free to post your findings on #deep-learning-questions-and-debugging to see if others have gotten similar differences!

This next part is pretty open-ended, so feel free to take some creative liberties. We want to use ChatGPT to filter its own outputs. What kind of prompt would you use to combine the two generated responses into a better, third output?

In [ ]:
combine_system = ...
combine_prompt = ...

response = ...
combined_response = ...

In practice, the strategy of using the large language model to refine itself is used quite often! While there are many possible ways to get the prompts to do what you want, here are a few guidelines provided by OpenAI: [cookbook](https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md)



## Open-Ended Exploration (Optional)

We hope this tutorial was helpful in understanding the ins and outs of OpenAI's ChatGPT API.

There are definitely many more things we can do with this dataset, but we would love to see what you can think of.

In the remaining part of this assignment, feel free to come up with a task and post your findings on Slack under #deep-learning-questions-and-debugging.

Here are a few ideas:

1. Look for menu items across restaurants and find the best restaurant for a specific menu item.

2. Compare and contrast two businesses given their reviews.

3. Find the most controversial dish!

## Final Step

Before submitting this assignment, remember to remove your OpenAI API key from the beginning of this assignment 😀.